## Setting up 

In [1]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool, CustomJS, LabelSet
import numpy as np

output_notebook()

Loading BokehJS ...

## Defining Variables

In [2]:
# long string variables
twentysix = [1,2,3,4,5,6,7,8,9,10,11,12,
             13,14,15,16,17,18,19,20,21,22,23,24,25,26]
alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m',
            'n','o','p','q','r','s','t','u','v','w','x','y','z',
            'a','b','c','d','e','f','g','h','i','j','k','l','m',
            'n','o','p','q','r','s','t','u','v','w','x','y','z']


# aesthetic variables
x_offset = -4.5    # letter annotation offset from circle edge
y_offset = -7    # letter annotation offset from circle edge

seg_color = 'gold'    # segment color
line_width = 3.0    # segment line width
seg_alpha = 0.6    # segment alpha

circ_color = 'gold'    # circle color
circ_size = 23    # circle size
circ_alpha = 0.4    # circle alpha
hov_color = 'gold'    # hover color for circles
hov_alpha = 0.8    # hover alpha for circles

font = 'Garamond'


# some points
x = np.array([1,6])
x = np.repeat(x,26)
y = twentysix + twentysix

# Rotor Links - Rotor 1

In [3]:
# establish links
links = {
    0:[30],
    1:[36],
    2:[38],
    3:[31],
    4:[37],
    5:[32],
    6:[29],
    7:[42],
    8:[47],
    9:[51],
    10:[39],
    11:[45],
    12:[40],
    13:[48],
    14:[50],
    15:[33],
    16:[49],
    17:[46],
    18:[44],
    19:[41],
    20:[26],
    21:[34],
    22:[27],
    23:[43],
    24:[28],
    25:[35],
        
    26:[20],
    27:[22],
    28:[24],
    29:[6],
    30:[0],
    31:[3],
    32:[5],
    33:[15],
    34:[21],
    35:[25],
    36:[1],
    37:[4],
    38:[2],
    39:[10],
    40:[12],
    41:[19],
    42:[7],
    43:[23],
    44:[18],
    45:[11],
    46:[17],
    47:[8],
    48:[13],
    49:[16],
    50:[14],
    51:[9]
}

## Create Graph and Add Features

In [4]:
# create graph
p = figure(width=400, height=800, tools="",
           toolbar_location=None,title='Hover over points')

# add letter labels
lsource = ColumnDataSource(data=dict(x = x, y = y, names = alphabet))
labels = LabelSet(x='x', y='y', text='names', level='annotation',
                  x_offset=x_offset, y_offset=y_offset, source=lsource,
                  render_mode='canvas', text_font=font, 
                  text_color='black', text_font_style='bold')
p.add_layout(labels)

# add glyphs
source = ColumnDataSource({'x0': [], 'y0': [], 'x1': [], 'y1':[]})
sr = p.segment(x0='x0',y0='y0',x1='x1', y1='y1', color=seg_color, 
               alpha=seg_alpha, line_width=line_width, source=source,)
cr = p.circle(x, y, color=circ_color, size=circ_size, alpha=circ_alpha, 
              hover_color=hov_color, hover_alpha=hov_alpha)


# JS for hover tool
code = """
var links = %s;
var data = {'x0': [], 'y0': [], 'x1': [], 'y1': []};
var cdata = circle.get('data');
var indices = cb_data.index['1d'].indices;
for (i=0; i < indices.length; i++) {
    ind0 = indices[i]
    for (j=0; j < links[ind0].length; j++) {
        ind1 = links[ind0][j];
        data['x0'].push(cdata.x[ind0]);
        data['y0'].push(cdata.y[ind0]);
        data['x1'].push(cdata.x[ind1]);
        data['y1'].push(cdata.y[ind1]);
    }
}
segment.set('data', data);
""" % links

# Call JS and Add HoverTool
callback = CustomJS(args={'circle':cr.data_source, 
                          'segment':sr.data_source}, code=code)
p.add_tools(HoverTool(tooltips=None,callback=callback,renderers=[cr]))

# Display Graph

In [5]:
show(p)

In [6]:
print(x)
print(y)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]


In [7]:
d = {'xs': x, 'ys':y}
source = ColumnDataSource(d)

In [8]:
p.multi_line('xs','ys', source=source, color='firebrick', line_width=3)

GlyphRenderer(id='b8c253ab-dfd1-44f7-80c7-f684b565eaf5', ...)

In [13]:
q = figure()
q.circle(x='xs', y='ys', source=source)
show(q)

# Figure out a way to show the other links all the time and only highlight one with the hover.  Need to look into JS link.  See http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html#customjs-for-hover